In [8]:
from ipynb.fs.full.Equity_Scrape import download_equity
from ipynb.fs.full.Stock_Scrape1 import download_stocks
from ipynb.fs.full.CorporateActions_Scrape1 import download_corporate_actions
from ipynb.fs.full.BhavCopy_Scrape import download_bhavcopy
from ipynb.fs.full.Deliverable_Scrape import download_deliverables
from ipynb.fs.full.ConvertBhavToStock import convertBhavCopyToStock
from ipynb.fs.full.Index_Scrape import download_index
from ipynb.fs.full.RiskFreeRate_Scrape import download_risk_free_rate
from ipynb.fs.full.Revenue_Profit_Scrape import download_revenue_profit

from ipynb.fs.full.Data_Cleaning import data_cleaning
from ipynb.fs.full.CorporateActions import apply_corporate_actions
from ipynb.fs.full.Feature_Creation1 import calculate_beta, calculate_alpha, add_risk_free_column,create_lower_upper_bands, calculate_band_area, create_dividend, create_index, add_next_day_columns
from ipynb.fs.full.Revenue import create_eps_pe_ratio_revenue_income_expenditure_net_profit
from ipynb.fs.full.GrowthRate import find_gain_loss, close_price_as_percent_of_LV_HV_BA, sequential_increase, sequential_decrease, sequential_increase_percentage, sequential_decrease_percentage, max_min_avg_of_sequential_data, sequential_increase_decrease, generate_dictionary_for_quarterwise_data, generate_dictionary_for_quarterwise_growthrate_data, update_growthrate_for_quarterwise_data, quarter_wise_growthrate

import pandas as pd 
import numpy as np
import os
import time

In [9]:
path = os.getcwd()
datapath = os.path.join(path,"Data")
datapath

'C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data'

In [10]:
companies = {"500112" : "SBIN" ,
"500325" : "RELIANCE INDUSTRIES LTD",
"532540" : "TATA CONSULTANCY SERVICES LTD" ,
"500209" : "INFOSYS LTD", 
"532174" : "ICICI BANK LTD", 
"507685" : "WIPRO LTD", 
"530965" : "INDIAN OIL CORPORATION LTD", 
"500182" : "HERO MOTOCORP LTD", 
"532210" : "CITY UNION BANK LTD", 
"500180" : "HDFC Bank Ltd"}

In [11]:
security_id = list(companies.keys())
company_name = list(companies.values())

In [12]:
print(security_id)
print(company_name)

['500112', '500325', '532540', '500209', '532174', '507685', '530965', '500182', '532210', '500180']
['SBIN', 'RELIANCE INDUSTRIES LTD', 'TATA CONSULTANCY SERVICES LTD', 'INFOSYS LTD', 'ICICI BANK LTD', 'WIPRO LTD', 'INDIAN OIL CORPORATION LTD', 'HERO MOTOCORP LTD', 'CITY UNION BANK LTD', 'HDFC Bank Ltd']


In [6]:
download_equity()

Equity file already exists


In [7]:
download_bhavcopy()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [8]:
download_deliverables()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [9]:
bhav_df_today = pd.read_csv(os.path.join(datapath,"BhavCopies/01032021.csv"))
deliverables_df_today = pd.read_csv(os.path.join(datapath,"Deliverables/01032021.csv"))
df_stocks_today = convertBhavCopyToStock(bhav_df_today,deliverables_df_today)
df_stocks_today.to_csv(os.path.join(datapath,"BhavConvertedToStock/01032021.csv"),index=None)

In [10]:
download_index()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [11]:
download_risk_free_rate()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


C:\Users\venu\Desktop\Stock Market Analysis\RiskFreeRate_Scrape.ipynb:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        return driver\n",


In [14]:
%%time
for i in range(9, len(security_id)):
    print(security_id[i])
    start_time = time.time()
    print(start_time)
#     download_corporate_actions(security_id[i])
    download_revenue_profit(security_id[i],company_name[i])
    stock_df = download_stocks(security_id[i])
    
    modified_stock_path = os.path.join(os.getcwd(),"Data\Stock\mod" + security_id[i] + ".csv")
    modified_stock = pd.read_csv(modified_stock_path) if os.path.exists(modified_stock_path) else pd.DataFrame()
    
    index_df = pd.read_csv(os.path.join(datapath,"Index.csv"))
    corporate_df = pd.read_csv(os.path.join(datapath,"Corporate Actions\\" + "Corporate_Actions_" + security_id[i] + ".csv"))
    revenue_df = pd.read_csv(os.path.join(datapath,"Revenue/" + security_id[i] + ".csv"))
    
    stock_df = data_cleaning(stock_df,index_df)
    stock_df = apply_corporate_actions(stock_df,corporate_df)
    stock_df = add_risk_free_column(stock_df)
    
    create_index()
    
    stock_df = calculate_beta(stock_df)
    stock_df = calculate_alpha(stock_df)
    stock_df = create_dividend(corporate_df,stock_df)
    stock_df = create_eps_pe_ratio_revenue_income_expenditure_net_profit(revenue_df,stock_df)
    stock_df = create_lower_upper_bands(stock_df, modified_stock.tail(1))
    stock_df = add_next_day_columns(stock_df)
    
    modified_stock = modified_stock.append(stock_df,ignore_index=True)
    modified_stock = modified_stock.drop_duplicates(subset = ["Date"], keep = "first")
    modified_stock.to_csv(modified_stock_path,index=None)
    
    modified_GRstock_path = os.path.join(os.getcwd(),"Data\Stock\mod" + security_id[i] +"GR.csv")
#     modified_stockGR = pd.read_csv(modified_GRstock_path) if os.path.exists(modified_GRstock_path) else pd.DataFrame()
    mstock_df = pd.read_csv(modified_stock_path)
    
    mstock_df['Alpha'] = pd.to_numeric(mstock_df['Alpha'], errors='coerce')
    mstock_df['Beta'] = pd.to_numeric(mstock_df['Beta'], errors='coerce')
    
    stock = find_gain_loss(mstock_df)
    stock = sequential_increase(stock)
    stock = sequential_decrease(stock)
    stock = sequential_increase_percentage(stock)
    stock = sequential_decrease_percentage(stock)
    stock = sequential_increase_decrease(stock)
    
    
    cols = ["Revenue","Dividend","Income","Expenditure","Net Profit","EPS"]
    for col in cols:
        try:
            stock = quarter_wise_growthrate(stock, col)
        except:
            pass
    
    stock = close_price_as_percent_of_LV_HV_BA(stock)
    
    if (os.path.exists(modified_GRstock_path)):
        os.remove(modified_GRstock_path)
        stock.to_csv(os.path.join(datapath,"Stock\\"+"mod"+security_id[i]+"GR.csv"),index=None)
    else:
        stock.to_csv(os.path.join(datapath,"Stock\\"+"mod"+security_id[i]+"GR.csv"),index=None)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    print(security_id[i], "completed")
    

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


500180
1614702077.5471733
 


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


 
--- 342.93392300605774 seconds ---
500180 completed
Wall time: 5min 42s
